## Check GPU's availability

In [ ]:
## The usual imports
import torch
## print out the pytorch version used
print(f"torch version: {torch.__version__}, GPU support: {torch.cuda.is_available()}")

## Load Trained Model to Test

In [2]:
import pytorch_lightning as pl
from torchmetrics.classification import ConfusionMatrix
from resnet_classifier import *

exp_root = "../../_exp/"
task_name = exp_root + "ClassAB/"
num_classes = 2
ckpt_name = "/models/last.ckpt"  # Grayscale

if (False): # testing anamal classification
    task_name = exp_root + "Dataset/"
    num_classes = 3
    ckpt_name = "/models/lightning_logs/version_2/checkpoints/epoch=9-step=650.ckpt"

model_name = task_name + ckpt_name
model = ResNetClassifier.load_from_checkpoint(model_name, dataset_root=task_name)
model.eval();

In [ ]:
trainer = pl.Trainer(default_root_dir=task_name) # (fast_dev_run=3)
# trainer.predict(model, model.test_dataloader());  # result (preds & gts) are stored in model.output_preds & output_gts
trainer.test(model);

## Detailed Study of Model's Classification Error

In [ ]:
y_gts   = torch.cat(model.output_gts).detach().cpu()
y_preds = torch.cat(model.output_preds).detach().cpu()

# study confusion matrix
y_hat  = torch.argmax(y_preds, dim=1)
confmat = ConfusionMatrix(task = 'multiclass', num_classes=model.num_classes)
print(confmat(y_hat, y_gts))

In [ ]:
from utils import *

testset = model.test_dataloader().dataset
for cls_id in range(model.num_classes):
    collect_error_cases(y_hat, y_gts, testset, cls_id, show=True)

In [ ]:
testloader = model.test_dataloader()

imgs, labels = next(iter(testloader))
show_images(imgs, labels)